# Testing

In [1]:
import duckdb

In [2]:
circuits = duckdb.sql("""SELECT * FROM 'data/*_circuit_profiles.csv'""").df()

In [3]:
drivers = duckdb.sql("""SELECT * FROM 'data/*_driver_profiles.csv'""").df()

In [5]:
circuits

,year,event,location,session,real_altitude,lap_length,telemetry_source,avg_speed,top_speed,braking_events,low_pct,med_pct,high_pct,slow_corners,medium_corners,fast_corners,chicanes,air_temp_avg,track_temp_avg,rain_detected
0,2024,Bahrain Grand Prix,Sakhir,FP1,8.0,5357.197222,fastf1,206.997167,310.0,11,0.124646,0.286119,0.589235,3,5,7,6,19.373171,34.215854,False
1,2024,Bahrain Grand Prix,Sakhir,FP2,8.0,5342.252778,fastf1,213.737952,316.0,10,0.114458,0.268072,0.617470,3,5,7,6,17.247561,22.098780,False
2,2024,Bahrain Grand Prix,Sakhir,FP3,8.0,5390.449722,fastf1,212.495652,316.0,10,0.124638,0.255072,0.620290,3,5,7,6,20.259756,32.013415,False
3,2024,Bahrain Grand Prix,Sakhir,Q,8.0,5369.465278,fastf1,215.886567,319.0,15,0.119403,0.259701,0.620896,3,4,8,7,18.049351,21.514286,False
4,2024,Bahrain Grand Prix,Sakhir,R,8.0,5356.580278,fastf1,208.853868,301.0,7,0.120344,0.269341,0.610315,3,5,7,6,18.227389,23.652866,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,2025,Austrian Grand Prix,Spielberg,FP2,681.0,4293.358611,fastf1,238.681034,322.0,9,0.064655,0.150862,0.784483,1,2,7,2,25.467089,36.336709,False
172,2025,Austrian Grand Prix,Spielberg,FP3,681.0,4291.315556,fastf1,244.646552,321.0,10,0.060345,0.129310,0.810345,1,2,7,2,24.560759,36.300000,False
173,2025,Austrian Grand Prix,Spielberg,Q,681.0,4298.454167,fastf1,242.557447,324.0,8,0.068085,0.136170,0.795745,1,2,7,3,27.773864,47.196591,False
174,2025,Austrian Grand Prix,Spielberg,R,681.0,4286.687500,fastf1,225.078189,319.0,7,0.094650,0.213992,0.691358,1,2,7,3,30.347826,49.166460,False
